# component

> TODO fill in description

In [1]:
#| default_exp component

In [2]:
#| hide
from nbdev.showdoc import *; import nbdev; nbdev.nbdev_export()

In [3]:
#|export
import asyncio
from abc import ABC, abstractmethod
from typing import Type, Optional, Callable, Any
from enum import Enum

from fbdev.utils import AttrContainer
from fbdev.packet import Packet

NameError: name 'BasePort' is not defined

In [ ]:
#|hide
import fbdev

In [ ]:
#|hide
show_doc(fbdev.component.PortType)

---

### PortType

>      PortType (value, names=None, module=None, qualname=None, type=None,
>                start=1)

*An enumeration.*

In [ ]:
#|export
class PortType(Enum):
    INPUT = ("input", True)
    OUTPUT = ("output", False)
    CONFIG = ("config", True)
    SIGNAL = ("signal", False)
    
    def __init__(self, label, is_input_port):
        self._label = label
        self._is_input_port = is_input_port
        
    @property
    def label(self): return self._label
    @property
    def is_input_port(self): return self._is_input_port

In [ ]:
#|hide
show_doc(fbdev.component.PortSpec)

---

### PortSpec

>      PortSpec (dtype=typing.Optional[typing.Type], data_validator=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PortSpec:
    def __init__(self, dtype=Optional[Type], data_validator=None):
        self._name = None
        self._port_type = None
        self._is_input_port = None
        self._dtype = dtype
        self._data_validator = data_validator
            
    @property
    def name(self): return self._name
    @property
    def port_type(self): return self._port_type
    @property
    def is_input_port(self): return self._is_input_port
    @property
    def dtype(self): return self._dtype
    @property
    def data_validator(self): return self._data_validator
    
    @property
    def has_dtype(self): return self._dtype is not None
    @property
    def has_data_validator(self): return self._data_validator is not None
    
    def __str__(self) -> str:
        return f"{self.port_type}.{self.name}"
    
    def __repr__(self) -> str:
        return str(self)

In [ ]:
#|hide
show_doc(fbdev.component.ConfigPortSpec)

---

### ConfigPortSpec

>      ConfigPortSpec (dtype=typing.Optional[typing.Type], data_validator=None,
>                      is_optional=False, default=None, default_is_None=False)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class ConfigPortSpec(PortSpec):
    def __init__(self, dtype=Optional[Type], data_validator=None, is_optional=False, default=None, default_is_None=False):
        super().__init__(dtype, data_validator)
        self._is_optional = is_optional
        
        if default is not None and default_is_None:
            raise RuntimeError("default cannot be None if default_is_None is True.")
        if default is not None or default_is_None:
            self._default = default
            
        if self.is_optional and self.has_default:
            raise RuntimeError("Config port {self.name} cannot have both be optional and have a default value.")
            
    @property
    def name(self): return self._name
    @property
    def dtype(self): return self._dtype
    @property
    def data_validator(self): return self._data_validator
    @property
    def is_optional(self): return self._is_optional
    @property
    def default(self):
        if not self.has_default: raise RuntimeError(f"Config port {self.name} does not have a default value.")
        return self._default
        
    @property
    def has_default(self): return hasattr(self, '_default')

In [ ]:
#|hide
show_doc(fbdev.component.PortTypeSpec)

---

### PortTypeSpec

>      PortTypeSpec (**port_specs)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PortTypeSpec(AttrContainer):
    def __init__(self, **port_specs):
        super().__init__({}, obj_name=type(self).__name__)
        self._port_type = None
        self._is_input_port = None
        for port_name, port_spec in port_specs.items():
            if not isinstance(port_spec, PortSpec):
                raise TypeError(f"{PortSpec.__name__} {port_name} is not of type {PortSpec.__name__}.")
            self._set(port_name, port_spec)
            port_spec._name = port_name
            
    @property
    def port_type(self): return self._port_type
    @property
    def is_input_port(self): return self._is_input_port
            
    def _initialise(self, port_type):
        self._port_type = port_type
        self._is_input_port = port_type in ['input', 'config']
        
        for port_name in self:
            if port_type != 'config' and type(self[port_name]) == ConfigPortSpec:
                raise ValueError(f"Invalid port spec {ConfigPortSpec.__name__} for port type {port_type}.")
            if port_type == 'config' and type(self[port_name]) != ConfigPortSpec:
                raise ValueError(f"Invalid port spec {port_name} for port type {port_type}.")
            self[port_name]._port_type = port_type
            self[port_name]._is_input_port = self._is_input_port
            
    def __str__(self) -> str:
        return f"{self.port_type} : {[port_spec.name for port_spec in self.values()]}"
    
    def __repr__(self) -> str:
        return str(self)


In [ ]:
#|hide
show_doc(fbdev.component.PortSpecCollection)

---

### PortSpecCollection

>      PortSpecCollection (input:fbdev.component.PortTypeSpec=None,
>                          output:fbdev.component.PortTypeSpec=None,
>                          config:fbdev.component.PortTypeSpec=None,
>                          signal:fbdev.component.PortTypeSpec=None)

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class PortSpecCollection:
    def __init__(self, input:PortTypeSpec=None, output:PortTypeSpec=None, config:PortTypeSpec=None, signal:PortTypeSpec=None):
        self._input = input or PortTypeSpec()
        self._input._initialise('input')
        
        self._output = output or PortTypeSpec()
        self._output._initialise('output')
        
        self._config = config or PortTypeSpec()
        self._config._initialise('config')
        
        self._signal = signal or PortTypeSpec()
        self._signal._initialise('signal')
    
    @property
    def input(self): return self._input
    @property
    def output(self): return self._output
    @property
    def config(self): return self._config
    @property
    def signal(self): return self._signal
    
    def __iter__(self):
        return iter([self.input, self.output, self.config, self.signal])
    
    def __str__(self) -> str:
        lines = []
        for port_type_spec in self:
            lines.append(f"{port_type_spec.port_type}:")
            for port_name, port_spec in port_type_spec.items():
                lines.append(f"  {str(port_spec.name)}")
        return "\n".join(lines)


In [ ]:
#|hide
show_doc(fbdev.component.BasePort)

---

### BasePort

>      BasePort (name:str, port_type:str,
>                component_process:fbdev.component.BaseComponent, dtype:Type,
>                data_validator:Callable[[Any],bool])

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class BasePort:
    def __init__(self, port_spec:PortSpec, parent):
        self._name = port_spec.name
        self._port_type = port_spec.port_type
        self._parent = parent
        self._dtype = port_spec.dtype
        self._data_validator = port_spec.data_validator
            
    async def _validate_packet_dtype(self, packet):
        if self.dtype is not None:
            if self.dtype != packet.dtype:
                raise TypeError("Packet is of type {packet.dtype}, but should be of type {self.dtype}.")
            
    async def _validate_packet(self, packet):
        if self.data_validator is not None:
            payload = await packet.consume()
            if not self.data_validator(payload):
                #TODO logging in BasePort._validate_packet_data
                #TODO proper exceptions in BasePort._validate_packet_data
                #TODO unit test BasePort._validate_packet_data
                raise RuntimeError("Packet payload failed data validation.")
            return Packet(payload)
        return packet

    @property
    def name(self): return self._name    
    @property
    def port_type(self): return self._port_type
    @property
    def idx(self): return self._idx
    @property
    def parent(self): return self._parent
    @property
    def dtype(self): return self._dtype
    @property
    def data_validator(self): return self._data_validator
    
    def __str__(self) -> str:
        return f"{self.port_type}.{self.name}"
    
    def __repr__(self) -> str:
        return str(self)

In [ ]:
#|hide
show_doc(fbdev.component.InputPort)

---

### InputPort

>      InputPort (name:str, port_type:str,
>                 component_process:fbdev.component.BaseComponent, dtype:Type,
>                 data_validator:Callable[[Any],bool])

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class InputPort(BasePort):
    def __init__(self, port_spec:PortSpec, parent):
        super().__init__(port_spec, parent)
        self._packet = None
        self._packet_waiting = asyncio.Event() # Set by a Node, when there is an edge waiting to sent to the packet
        self._requesting_packet = asyncio.Event() # Set when the port is requesting a package
        self._received_packet = asyncio.Event() # Set when the port has returned a package to the component process
        
    def _load_packet(self, packet):
        if self._packet is not None:
            raise RuntimeError(f"Port {self.parent.__class__.__name__}.{self.name} is already receiving a packet.")
        if not self._requesting_packet.is_set():
            raise RuntimeError(f"Port {self.parent.__class__.__name__}.{self.name} is not requesting a packet.")
        self._packet = packet
        self._received_packet.set()
        
    async def receive(self):
        self._requesting_packet.set()
        await self._received_packet.wait()
        self._requesting_packet.clear()
        self._received_packet.clear()
        packet = self._packet
        self._packet = None
        packet = await self._validate_packet(packet)
        return packet
    
    async def receive_payload(self):
        packet = await self.receive()
        return await packet.consume()

In [ ]:
#|hide
show_doc(fbdev.component.OutputPort)

---

### OutputPort

>      OutputPort (name:str, port_type:str,
>                  component_process:fbdev.component.BaseComponent, dtype:Type,
>                  data_validator:Callable[[Any],bool])

*Initialize self.  See help(type(self)) for accurate signature.*

In [ ]:
#|export
class OutputPort(BasePort):
    def __init__(self, port_spec:PortSpec, parent):
        super().__init__(port_spec, parent)
        self._packet = None
        self._edge_available = asyncio.Event() # Set by a Node, when there is an edge available for being sent a packet
        self._ready_to_unload = asyncio.Event() # Set when the port is requesting to send out a packet
        self._package_sent = asyncio.Event() # Set when the port has returned a package to the component process
        
    def _unload_packet(self):
        if self._packet is None:
            raise RuntimeError(f"Port {self.parent.__class__.__name__}.{self.name} does not have a packet to send.")
        if not self._ready_to_unload.is_set():
            raise RuntimeError(f"Port {self.parent.__class__.__name__}.{self.name} is not ready to send a packet.")
        packet = self._packet
        self._packet = None
        self._package_sent.set()
        return packet
        
    async def put(self, packet):
        if type(packet) != Packet: raise TypeError(f"`packet` must be of type {Packet.__name__}.")
        packet = await self._validate_packet(packet)
        self._packet = packet
        self._ready_to_unload.set()
        await self._package_sent.wait()
        self._ready_to_unload.clear()
        self._package_sent.clear()
        
    async def put_payload(self, packet_payload):
        packet = Packet(packet_payload)
        await self.put(packet)

In [ ]:
#|hide
show_doc(fbdev.component.PortCollection)

In [ ]:
#|export
class PortCollection(AttrContainer):
    def __init__(self, port_spec_collection:PortSpecCollection, parent):
        super().__init__({}, obj_name=f"{parent.__class__.__name__}.ports")
        self._parent = parent
        for port_type_spec in port_spec_collection:
            self._set(port_type_spec.port_type, AttrContainer({}, obj_name=f"{parent.__class__.__name__}.{port_type_spec.port_type}"))
            self._attach_ports(port_type_spec, self.ports[port_type_spec.port_type], port_type_spec.is_input_port)

    def _attach_ports(self, port_type_spec, port_type_container, is_input_port):
        for port_name, port_spec in port_type_spec.items():
            if is_input_port: port = InputPort(port_name, port_spec.port_type, self._parent, port_spec.dtype, port_spec.data_validator)
            else: port = OutputPort(port_name, port_spec.port_type, self._parent, port_spec.dtype, port_spec.data_validator)
            self[port_type_spec.port_type]._set(port_name, port)
    
    @property
    def input(self): return self._input
    @property
    def output(self): return self._output
    @property
    def config(self): return self._config
    @property
    def signal(self): return self._signal
    
    def __iter__(self):
        return iter([self.input, self.output, self.config, self.signal])
    
    def __str__(self) -> str:
        lines = []
        for port_type_spec in self:
            lines.append(f"{port_type_spec.port_type}:")
            for port_name, port_spec in port_type_spec.items():
                lines.append(f"  {str(port_spec.name)}")
        return "\n".join(lines)

In [ ]:
#|hide
show_doc(fbdev.component.BaseComponent)

---

### BaseComponent

>      BaseComponent ()

*Helper class that provides a standard way to create an ABC using
inheritance.*

In [ ]:
#|export
class BaseComponent(ABC):
    port_specs = PortSpecCollection(
        input=PortTypeSpec(),
        output=PortTypeSpec(),
        config=PortTypeSpec(),
        signal=PortTypeSpec(),
    )
    
    def __init_subclass__(cls, **kwargs):
        """Prevents any subclass from defining an __init__ that accepts any argument other than 'self'."""
        super().__init_subclass__(**kwargs)
        init_method = cls.__dict__.get('__init__', None)
        if init_method:
            from inspect import signature
            sig = signature(init_method)
            params = tuple([p.name for p in sig.parameters.values()])
            if params != ('self',):
                raise TypeError(f"{cls.__name__}.__init__() must only accept 'self' as an argument.")
    
    def __init__(self):
        self.ports = PortCollection(self.port_specs, self)
        self.config = AttrContainer({}, obj_name="Component.config")
        for config_port_name, config_port_spec in self.port_specs.config.items():
            if config_port_spec.has_default:
                self.set_config(config_port_name, config_port_spec.default)
            
    async def update_config(self):
        for port in self.config_inputs.values():
            if context.input_edge_has_packet[port.name]:
                self.config._set(port.name, await port.receive())
                    
    def set_config(self, name, value): 
        if self.port_specs.config[name].has_dtype and type(value) != self.port_specs.config[name].dtype:
            raise TypeError(f"Config value {value} is not of type {self.port_specs.config[name].dtype}, in config port {name}.")
        if self.port_specs.config[name].has_data_validator and not self.port_specs.config[name].data_validator(value):
            raise ValueError(f"Config value {value} is not valid for config port {name}, in config port {name}.")
        if name not in self.port_specs.config.keys():
            raise ValueError(f"Config port {name} is not a valid config port for component {self.__class__.__name__}.")
        self.config._set(name, value)
        
    def check_configured(self) -> bool:
        configured = True
        for config_port_name, config_port_spec in self.port_specs.config.items():
           if not config_port_name in self.config and not config_port_spec.is_optional:
               configured = False
               break
        return configured
    
    @abstractmethod
    async def execute(self):
        pass

In [ ]:
class MyComponent(BaseComponent):
    port_specs = PortSpecCollection(
        input=PortTypeSpec(
            in1=PortSpec(dtype=str, data_validator=lambda x: x.endswith("world")),
        ),
        output=PortTypeSpec(
            out1=PortSpec(dtype=str),
        ),
        config=PortTypeSpec(
            greeting=ConfigPortSpec(dtype=str, default="hello"),
            append=ConfigPortSpec(dtype=str, default="!"),
        ),
        signal=PortTypeSpec(),
    )
    
    async def execute(self):
        data = await self.ports.input.in1.receive()
        data_payload = await data.consume()
        await self.ports.output.out1.put(Packet(f"{self.config.greeting} {data_payload}{self.config.append}"))
        
comp = MyComponent()

async def packet_sender():
    await comp.ports.input.in1._requesting_packet.wait()
    comp.ports.input.in1._load_packet(Packet("world"))
async def packet_receiver():
    await comp.ports.output.out1._ready_to_unload.wait()
    packet = comp.ports.output.out1._unload_packet()
    packet_payload = await packet.consume()
    print(packet_payload)

await asyncio.gather(
    asyncio.create_task(packet_sender()),
    asyncio.create_task(comp.execute()),
    asyncio.create_task(packet_receiver()),
)

comp.set_config("greeting", "goodbye")
comp.set_config("append", "?")

await asyncio.gather(
    asyncio.create_task(packet_sender()),
    asyncio.create_task(comp.execute()),
    asyncio.create_task(packet_receiver()),
);

hello world!
goodbye world?
